# Download data via API

In [1]:
import pandas as pd
from sodapy import Socrata

MyAppToken='6v9nCt2L0Tftx31xuIhXQf5uv'
username='cz2527@columbia.edu'
password='Tools002'
client = Socrata('data.cityofnewyork.us',
                 MyAppToken,
                 username=username,
                 password=password)

results = client.get("8end-qv57",limit=3000)
results_df = pd.DataFrame.from_records(results)


In [2]:
len(results_df)

2227

In [69]:
results_df

community_board            end_date_time                   event_agency  \
0               11,   2018-12-05T23:00:00.000               Parks Department   
1               64,   2018-12-05T23:59:00.000               Parks Department   
2               02,   2018-11-26T17:00:00.000               Parks Department   
3               81,   2018-11-29T20:00:00.000               Parks Department   
4               64,   2018-12-19T23:59:00.000               Parks Department   
5               14,   2018-12-23T17:00:00.000  Street Activity Permit Office   
6               13,   2018-12-02T11:30:00.000               Parks Department   
7               64,   2018-12-02T13:00:00.000               Parks Department   
8               64,   2018-12-12T01:00:00.000               Parks Department   
9                5,   2018-12-02T23:59:00.000               Parks Department   
10               5,   2018-12-12T12:00:00.000               Parks Department   
11               7,   2018-12-23T18:00:00.000  Street Activity Permit Office   
12              64,   2018-11-26T17:30:00.000               Parks Department   
13              13,   2018-12-23T19:00:00.000               Parks Department   
14              64,   2018-11-27T23:59:00.000               Parks Department   
15               6,   2018-12-02T14:00:00.000               Parks Department   
16              11,   2018-12-19T23:00:00.000               Parks Department   
17              12,   2018-11-27T18:00:00.000  Street Activity Permit Office   
18               3,   2018-12-05T20:00:00.000               Parks Department   
19              64,   2018-12-05T13:00:00.000               Parks Department   
20               7,   2018-11-23T23:59:00.000               Parks Department   
21               8,   2018-12-16T23:59:00.000               Parks Department   
22               6,   2018-12-12T14:00:00.000               Parks Department   
23              64,   2018-11-28T15:00:00.000               Parks Department   
24               5,   2018-11-23T23:59:00.000               Parks Department   
25               5,   2018-11-29T13:00:00.000               Parks Department   
26              07,   2019-06-30T23:59:00.000               Parks Department   
27              64,   2018-12-19T23:59:00.000               Parks Department   
28              64,   2018-12-19T23:59:00.000               Parks Department   
29              10,   2018-12-19T14:00:00.000               Parks Department   
...              ...                      ...                            ...   
2197            11,   2018-12-22T23:00:00.000               Parks Department   
2198             5,   2018-12-12T20:00:00.000  Street Activity Permit Office   
2199             3,   2018-12-22T09:00:00.000               Parks Department   
2200             2,   2018-12-22T20:00:00.000               Parks Department   
2201            15,   2018-12-14T20:00:00.000               Parks Department   
2202             2,   2018-12-22T20:00:00.000  Street Activity Permit Office   
2203             5,   2018-12-22T13:00:00.000               Parks Department   
2204             5,   2018-12-22T20:00:00.000  Street Activity Permit Office   
2205            64,   2018-12-21T19:00:00.000               Parks Department   
2206             8,   2018-12-22T23:59:00.000               Parks Department   
2207             5,   2018-12-11T20:00:00.000  Street Activity Permit Office   
2208            64,   2018-12-22T23:59:00.000               Parks Department   
2209            10,   2018-12-22T09:00:00.000               Parks Department   
2210         12, 7,   2018-12-22T16:00:00.000  Street Activity Permit Office   
2211             4,   2018-12-22T20:00:00.000               Parks Department   
2212            15,   2018-12-13T20:00:00.000               Parks Department   
2213            12,   2018-12-22T15:00:00.000  Street Activity Permit Office   
2214             7,   2018-12-22T23:59:00.000               Parks Department   
2215 

# Lowercase

In [ ]:
for i in range(len(results_df)):
    results_df['event_name'][i] = results_df['event_name'][i].lower()

# special events modification
close: closure, maintenance, construction


In [53]:
#results_df[results_df['event_type']=='Special Event']['event_name'].unique()

In [5]:
def change_type(results_df,keywords,event_type):
    for i in range(len(results_df)):
        if results_df.loc[i]['event_type']=='Special Event':
            for j in keywords:
                if j in results_df.loc[i]['event_name']:
                    results_df.loc[i]['event_type']=event_type

In [6]:
change_type(results_df,('Closure','closure','CLOSED'),'Closure')
change_type(results_df,('Construction','construction'),'Construction')
change_type(results_df,('Maintenance',),'Maintenance')
change_type(results_df,('Farmers Market',),'Farmers Market')
#change_type(results_df,('Market',),'Special Market')
change_type(results_df,('Celebration','celebration'),'Celebration')

# event_type reference

In [8]:
results_df['event_type'].unique()

array(['Construction', 'Closure', 'Special Event', 'Farmers Market',
       'Celebration', 'Maintenance', 'Sidewalk Sale', 'Miscellaneous',
       'Plaza Partner Event', 'Plaza Event', 'Religious Event',
       'Production Event', 'Parade', 'Special Market', 'Street Event',
       'Embargo', 'Rigging Permit', 'Athletic Race / Tour',
       'Theater Load in and Load Outs ', 'Shooting Permit',
       'Public Program / Exhibitions', 'Block Party'], dtype=object)

# Keyword search

In [56]:
def count_name(keyword,event_type=0):
    if event_type !=0:
        df=results_df[results_df['event_type']==event_type][results_df['event_name'].apply(lambda x: keyword in x)]
        result=df.groupby('event_name').size().sort_values(ascending=False)
        return result
#        return df.groupby('event_name').count()['community_board']
    else:
        df=results_df[results_df['event_name'].apply(lambda x: keyword in x)]
        result=df.groupby('event_name').size().sort_values(ascending=False)
        return result.index  

In [59]:
count_name('','Special Event')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


event_name
Art Lunchtime Tours                                                                 31
Model Aircraft Flying                                                               31
Flying Radio Controled Model Planes at Field                                        31
Holiday Live Tree Display                                                           31
Model Helicopter Flying                                                             31
Big Apple Circus                                                                    31
Juggling                                                                            26
Annual Tree Lighting Springfield Park                                               25
menorah lighting 3                                                                  22
Anshe Sholom Chabad Chanukah Menorah                                                22
Chanukah on the Park                                                                22
menora lighting                 